In [1]:
from KeRLas import Brain, TimedGymEnv, Trainer
from KeRLas.models import defaultQModel
from KeRLas.models import DirectDiffModel, NaiveQModel, LateralDiffModel, DQN
from KeRLas.policies import BoltzmannQPolicy
import numpy as np
import gym, math
from lunar_lander import LunarLander

#env = GymEnv(CartPoleEnv(), tlimit=300)
env = TimedGymEnv(LunarLander(), tlimit=500)

observation_width = env.observation_space.shape[0]
nactions = env.action_space.n


#for i in range(10):
#    next(brain.Memory.generate_samples(20))
#_ = next(brain.trainig_data_generator(mbsize))



Using TensorFlow backend.


Couldn't import dot_parser, loading of dot files will not be possible.
WARN: gym.spaces.Box autodetected dtype as <type 'numpy.float32'>. Please provide explicit dtype.


In [2]:
rlmodel = DirectDiffModel(defaultQModel(observation_width, nactions), 0.99)
policy = BoltzmannQPolicy(0.0001)
brain = Brain(rlmodel, policy, training_policies=map(BoltzmannQPolicy, [10.0, 5.0, 2.0, 1.0, 0.01, 0.001]))


In [3]:
class Callback:
    
    def onEpisodeBegin(self, env, agents, observations):
        env.render()

    def onStep(self, env, env_done, feedback):
        env.render()
        
    def onEpisodeEnd(self, env, record):
        env.render()

trainer = Trainer(env, brain, 0.00, 10000)   

from KeRLas import GymPlayer
p = GymPlayer(env, brain, callback=Callback())

mbsize = 50

for i in xrange(50):
    metrics = trainer.train(mbsize, 1000, 50)
    
    nsum = 0
    N = 5
    reward_sum = 0.0
    with brain.training(False):
        for _ in range(N):
            history = p.runEpisode()
            reward = sum([r for _,_,_,r,_ in history])
            #print reward, history[-1][3]
            reward_sum += reward
            #print len(history)
            nsum += len(history)
        print float(nsum)/N, reward_sum/N, metrics

TypeError: onEpisodeEnd() takes exactly 3 arguments (4 given)